In [1]:
import numpy as np
import random
import pygame

# (y,x)
grid_x = 10
grid_y = 10

y_bound = grid_y - 1
x_bound = grid_x - 1

grid = np.ones((grid_y, grid_x))
grid = np.negative(grid)

terminal_state = (9,9)
grid[terminal_state[0]][terminal_state[1]] = 50

start_y = np.random.randint(low=0, high=grid_y)
start_x = np.random.randint(low=0, high=grid_x)
current_state = (start_y, start_x)


print("Grid size: ", grid.shape)
print("Start state: ", current_state)
print(grid)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Grid size:  (10, 10)
Start state:  (8, 3)
[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. 50.]]


# Movement Code of the Environment

In [2]:
def move(state,action):
    # (y,x)
    switch = {
        0: up,
        1: right,
        2: down,
        3: left,
        4: pickup
    }

    func = switch.get(action, lambda: "Invalid action")
    new_state = func(state)
    return new_state

def valid_move(state):
    if state[0] > y_bound:
        state = (state[0]-1, state[1])
    elif state[0] < 0:
        state = (state[0]+1, state[1])
        
    if state[1] > x_bound:
        state = (state[0], state[1] - 1)
    elif state[1] < 0:
        state = (state[0], state[1] + 1)
    
    return state
    
def up(state):
    new_state = (state[0] - 1, state[1])
    new_state = valid_move(new_state)
    return new_state

def right(state):
    new_state = (state[0], state[1] + 1)
    new_state = valid_move(new_state)
    return new_state
    
def down(state):
    new_state = (state[0] + 1, state[1])
    new_state = valid_move(new_state)
    return new_state
    
def left(state):
    new_state = (state[0], state[1] - 1)
    new_state = valid_move(new_state)
    return new_state
    
def pickup(state):
    return state

# Actual transition of the environment based on action

In [3]:
def step(state, action):
    
    state_prime = move(state,action)
    
    reward = grid[state_prime[0]][state_prime[1]]
    
    if state_prime == terminal_state:
        done = True
    else:
        done = False
    
    return state_prime, reward, done

# Best action selection function

In [4]:
def best_action_value(state):
    best_action = None
    best_value  = float('-inf')

    for action in ACTION_SPACE:
        state_prime, reward, done = step(state, action) 
        print(state_prime, " - ", reward, " - ",  done)
        v = reward + gamma * V[state_prime]    
        
        if v > best_value:
            best_value = v
            best_action = action
    return best_action

In [25]:
ACTION_SPACE = list(range(0,4))
print("Action Space:")
print(ACTION_SPACE)
print(["Up", "Right", "Down", "Left", "Pickup"])


STATE_SPACE = np.zeros((grid_y, grid_x))
print("State Space:")
print(STATE_SPACE)

Reward = grid
V = np.zeros((grid_y, grid_x)) 
Pi = np.zeros((grid_y, grid_x))  
gamma = 0.9 
significant_improvement = 0.01 
print(Reward)

Action Space:
[0, 1, 2, 3]
['Up', 'Right', 'Down', 'Left', 'Pickup']
State Space:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. 50.]]


In [6]:
#for iteration in range(100):
#    for y in range(height):
#        for x in range(width): 
#            s = (y, x)
#            print("Position: ", s)
#            old_v = V[s]
#            action = best_action_value(s) 
#            print("Best action: ", action)
#            s_new, rew, done = step(s,action) 
#            V[s] = rew + gamma * V[s_new] 
#            Pi[s] = action
            
#print(V)
#print(Pi)

In [7]:
#temp = best_action_value(current_state)

In [24]:
done = False
reward_total = 0
iteration = 0


BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
 
pygame.init()

CONSTANT_SIZE = 50
height = grid.shape[0]
width = grid.shape[1]

size_y = (CONSTANT_SIZE * height) + height + 1
size_x = (CONSTANT_SIZE * width) + width + 1
screen_size = (size_y, size_x)

screen = pygame.display.set_mode(screen_size)

pygame.display.set_caption("Badworld")
 
session = True

clock = pygame.time.Clock()

print("Starting position: ", current_state)

while session:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            session = False
    
    if done == True or iteration == 20:
        #session = False
        break
    else:
        iteration += 1
 
    screen.fill(BLACK)
    
    for y in range(height):
        for x in range(width):                
            if y == current_state[0] and x == current_state[1]:
                colour = GREEN
            elif grid[y][x] == 50:
                colour = RED
            else:
                colour = WHITE
            
            rect_pos_x = x*(CONSTANT_SIZE+1)+1
            rect_pos_y = y*(CONSTANT_SIZE+1)+1
            rect = pygame.Rect(rect_pos_x, rect_pos_y , CONSTANT_SIZE, CONSTANT_SIZE)
            pygame.draw.rect(screen, colour, rect)    
            
    
    action = random.choice(ACTION_SPACE)
    obs, reward, done = step(current_state,action)
    current_state = obs
    reward_total = reward_total + reward
    
    print("Iteration", iteration, "- Reward:" ,reward_total)
    
    pygame.display.flip()
    pygame.time.wait(250)
    clock.tick(60)

pygame.quit()

print("Ending position: ", current_state)

Starting position:  (4, 7)
Iteration 1 - Reward: -1.0
Iteration 2 - Reward: -2.0
Iteration 3 - Reward: -3.0
Iteration 4 - Reward: -4.0
Iteration 5 - Reward: -5.0
Iteration 6 - Reward: -6.0
Iteration 7 - Reward: -7.0
Iteration 8 - Reward: -8.0
Ending position:  (4, 5)


In [9]:
pygame.quit()

In [10]:
#texting dreams

#font = pygame.font.Font('freesansbold.ttf', 10) 
#draw_text = True

#if draw_text == True:
#        text_pos_x = current_state[1]*(CONSTANT_SIZE+1)+1
#        text_pos_y = current_state[0]*(CONSTANT_SIZE+1)+1
#        rect = pygame.Rect(text_pos_x, text_pos_y , CONSTANT_SIZE, CONSTANT_SIZE)
#        text_word = "(" +  str(current_state[0]) + "; " + str(current_state[1]) + ")"
#        text_pos_x = current_state[1]*(CONSTANT_SIZE+1)+1
#        text_pos_y = current_state[0]*(CONSTANT_SIZE+1)+1
#        text = font.render(text_word, True, BLACK, WHITE) 
#        textRect = text.get_rect()  
#        textRect.center = (text_pos_x+(CONSTANT_SIZE/2), text_pos_y+(CONSTANT_SIZE/2)) 
#        screen.blit(text, textRect)